# 1. Naive Bayes Classification (나이브 베이즈 분류)

## 1-1. Bayes Theorem (베이즈 정리)

<img align="center" width="300" height="200" src="https://miro.medium.com/max/666/1*Pybqu8ks-6uNp2VrKQ-cGA.png">
<br/>
A: 원인, B: 결과<br/>
<br/>

- 사전확률과 사후확률 사이의 관계를 나타내는 정리
- 새로운 정보를 토대로 어떤 사건이 발생했다는 주장에 대한 신뢰도를 갱신해 나가는 방법
- 즉, '어떤 사건 A가 일어났다고 가정할 때 B라는 자료를 얻게 될 확률'에 대한 정보만 알고 있다면, 자료에 근거해 어떤 사건이 일어날 확률을 새로 계산할 수 있다는 뜻

\<ex> <br/>
- 어떤 사람이 병에 걸렸다고 진단 받았을 때, 진짜 병에 걸렸을 확률 <br/>
$\rightarrow$ A: 병에 걸림, B: 병에 걸렸다고 진단 받음

## 1-2. Naive Bayes Algorithm

- 지도학습
- 데이터가 각 클래스에 속할 확률을 계산하는 조건부 확률 기반의 분류 방법 <br/>
- Naive한 가정: 사건들이 상호 <b>독립적</b> 이라고 가정, 모든 변수들이 동등함 <br/>
<br/>
$P(A|B_1 \cap B_2\cap \ldots \cap B_n) = P(B_1|A)P(B_2|A) \ldots P(B_n|A)P(A) = P(A) \Pi_{i=1}^{n}P(B_i|A)$
<br/><br/>
독립성을 가정하면서 식이 비교적 간단해짐 == 모형이 단순해짐

## 1-3. 장단점

### 장점

- 간단하고 빠르며 효율적이다
- 노이즈와 결측치를 잘 처리한다
- train 시 데이터의 크기와 상관 없이 잘 동작한다
- 예측을 위한 추정 확률을 쉽게 얻을 수 있다.

### 단점

- 현실 데이터에는 "모든 feature의 중요성이 동등하고, 독립이다"라는 가정이 잘못된 경우가 훨씬 많다.
- numeric feature가 많은 데이터셋에는 이상적이지 않다.
- 추정된 확률이 예측된 클래스보다 덜 신뢰할만하다.

## 1-4. 보정

### Laplace Smoothing (라플라스 스무딩)

조건부 확률이 하나라도 0이라면 전체 확률이 0이 되는 상황이 발생함 <br/>
이럴 경우 값을 보정하기 위해 실제 관찰한 것보다 한 번씩 더 봤다고 가정하는 것이 라플라스 스무딩 기법 <br/>
즉, 확률이 0일 경우를 제외시키는 방법 <br/>
<br/>
$P_{LAP} = \frac{c(x)+1}{\Sigma x[c(x)+1]}$

### Underflow 방지

특정 확률이 0에 가까워질 수 있는 경우 확률에 로그를 취함으로써 언더플로우를 방지 <br/>
<br/>
$log[P(A|B)] = log[P(B|A)P(A)/P(B)]$

## 1-5. 예제

In [2]:
# import libraries
import pandas as pd
from sklearn.datasets import load_breast_cancer
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import GaussianNB # 나이브 베이즈 모델 중 가우시안 모델 사용
from sklearn.metrics import accuracy_score

In [3]:
# 데이터 불러오기
bc = load_breast_cancer()

# 데이터프레임으로 변환 (데이터 확인용으로 사용함)
df = pd.DataFrame(bc.data, columns=bc.feature_names)
df['target'] = pd.Series(bc.target)

df.head()

,mean radius,mean texture,mean perimeter,mean area,mean smoothness,mean compactness,mean concavity,mean concave points,mean symmetry,mean fractal dimension,...,worst texture,worst perimeter,worst area,worst smoothness,worst compactness,worst concavity,worst concave points,worst symmetry,worst fractal dimension,target
0,17.99,10.38,122.80,1001.0,0.11840,0.27760,0.3001,0.14710,0.2419,0.07871,...,17.33,184.60,2019.0,0.1622,0.6656,0.7119,0.2654,0.4601,0.11890,0
1,20.57,17.77,132.90,1326.0,0.08474,0.07864,0.0869,0.07017,0.1812,0.05667,...,23.41,158.80,1956.0,0.1238,0.1866,0.2416,0.1860,0.2750,0.08902,0
2,19.69,21.25,130.00,1203.0,0.10960,0.15990,0.1974,0.12790,0.2069,0.05999,...,25.53,152.50,1709.0,0.1444,0.4245,0.4504,0.2430,0.3613,0.08758,0
3,11.42,20.38,77.58,386.1,0.14250,0.28390,0.2414,0.10520,0.2597,0.09744,...,26.50,98.87,567.7,0.2098,0.8663,0.6869,0.2575,0.6638,0.17300,0
4,20.29,14.34,135.10,1297.0,0.10030,0.13280,0.1980,0.10430,0.1809,0.05883,...,16.67,152.20,1575.0,0.1374,0.2050,0.4000,0.1625,0.2364,0.07678,0


In [4]:
# train / test set으로 분리
X_train, X_test, y_train, y_test = train_test_split(bc.data, bc.target, test_size=0.3, random_state=42)

gnb = GaussianNB()

# 학습
gnb.fit(X_train, y_train)

# 예측
y_pred = gnb.predict(X_test)

In [5]:
# 정확도
print('Accuracy: ', round(accuracy_score(y_test, y_pred), 4)*100, '%')

Accuracy:  94.15 %


# 2. SVM (Support Vector Machine) 

## 2-1. SVM

- 지도학습 모델
- 데이터를 고차원 공간으로 사상시킨 후 support vector로 이루어진 초평면을 이용하여 선형 분류하는 __마진 기반__ 기계학습 모델
- 이때 계산량의 증가를 방지하기 위해서 커널함수 (kernel function) 사용

<img align="center" width="300" height="200" src="https://www.researchgate.net/profile/Muhammad-Awais-Bin-Altaf/publication/272520997/figure/fig2/AS:601593388998663@1520442449352/Motivation-behind-non-linear-SVM-classifier.png">

A의 경우, 쉽게 선형 분류가 가능 <br/>
<br/>
그렇다면, B의 경우는? <br/>
<br/>
$\rightarrow$ 고차원 공간으로 사상시켜서 초평면을 이용하여 선형 분류

## 2-2. Kernel Function (커널 함수)

- 데이터를 고차원 공간으로 mapping 시키는 함수
- 선형 분리 불가능 문제를 선형 분리 가능 문제로 변환해줌
- 직선이 파라미터를 찾기 제일 쉬운 함수이므로 가능한 linear-separable하게 변환하는 게 좋음

## 2-3. Support Vector (지지 벡터) / Maximum Margin (최대 마진)

__Support Vector__ <br/>
- 선형 분류의 경계 주변에 존재하는 데이터 포인트들 <br/>
<img align="left" width="300" height="300" src="https://learnopencv.com/wp-content/uploads/2018/07/support-vectors-and-maximum-margin.png">


__Maximum Margin__ <br/>
- 선형 분류를 가능하게 하는 직선은 무수히 많이 존재하지만, 그 중 마진을 최대로 하는 직선이 가장 이상적
- 마진을 최대로 하는 직선이 가장 이상적인 이유?
    - Outlier의 영향을 가장 덜 받기 때문(가장 Robust하기 때문) <br/>
<img align="center" width="500" height="300" src="https://static.packt-cdn.com/products/9781783555130/graphics/3547_03_07.jpg"> <br/>
- 직선의 방정식: $y=wx + b = 0$ => w와 b를 찾는 문제
- $wx^+ + b = 1$ => positive 영역의 margin line <br/>
- $wx^- + b = -1$ => negative 영역의 margin line <br/>
- 두 직선의 거리(Margin)를 구하려면: <br/>
$w(x^+ - x^-) = 2$ <br/>
$ x^+ - x^- =  \frac{2}{||w||}$ <br/>

## 2-4. Optimization Problem (최적화 문제)

- 목적함수: (너비 최대화) $max(\frac{2}{||w||}) \leftrightarrow min(\frac{1}{2}||w||^2)$ (벡터 w의 자기 자신과의 내적값 최소화)
- 제약조건: $y_i(wx_i+b) = 1$

### 라그랑주 승수법
- 제약식에 형식적인 라그랑주 승수(Lagrange Multiplier)를 곱한 항을 최적화하려는 목적식에 더하여 제약된 문제를 제약이 없는 문제로 바꾸는 기법 <br/>

$J(w, b, \alpha) = \frac{1}{2}w \centerdot w - \Sigma^n_{i=1} \alpha_i\{y_i(wx_i - b) - 1\}$ <br/>
$\alpha$ : Lagrange Multiplier <br/>
w, b에 대한 최소값 = 정점 => 미분했을 때 기울기가 0인 곳 <br/>
조건1: $\frac{\partial J(w, b, \alpha)}{\partial w} = 0$ <br/>
조건1: $\frac{\partial J(w, b, \alpha)}{\partial b} = 0$ <br/>
<br/>
$\rightarrow$ $w = \Sigma^n_{i=1}\alpha_i y_i x_i$ & $\Sigma^n_{i=1}\alpha_i y_i = 0$ <br/>
<br/>
자세한 내용은 <a href="https://untitledtblog.tistory.com/96">여기</a>를 참고

### Dual 문제
- Dual 문제로 변환하면 w를 없애고 training data에서 얻을 수 있는 것으로 바꿀 수 있음 <br/>
<br/>
$J(w, b, \alpha) = \frac{1}{2}w \centerdot w - \Sigma^n_{i=1} \alpha_i\{y_i(wx_i - b) - 1\}$ & $w = \Sigma^n_{i=1}\alpha_i y_i x_i$ & $\Sigma^n_{i=1}\alpha_i y_i = 0$ <br/>
$J(w, b, \alpha) = \frac{1}{2}w \centerdot w - \Sigma^n_{i=1}\alpha_i y_i wx_i - b \Sigma^n_{i=1}\alpha_i y_i + \Sigma^n_{i=1} \alpha_i$ <br/>
여기서 $\Sigma^n_{i=1}\alpha_i y_i wx_i = w \centerdot \Sigma^n_{i=1}\alpha_i y_i x_i = w \centerdot w$ <br/>
<br/>
$J(w, b, \alpha)$ 를 $Q(\alpha)$라고 한다면, <br/>
$Q(\alpha) = \Sigma^n_{i=1} \alpha_i - \frac{1}{2}\Sigma^n_{i=1}\Sigma^n_{j=1}\alpha_i \alpha_j y_i y_j x_i x_j$ <br/>
조건: $\Sigma^n_{i=1}\alpha_i y_i = 0$, $ \alpha \geq 0 $
<br/>
<br/>
- w와 b에 대한 최소값 = $\alpha$에 대한 최대값 $\rightarrow$ $Q(\alpha)$의 최대값을 구하면 됨<br/>

자세한 내용은 <a href="https://ratsgo.github.io/convex%20optimization/2018/01/25/duality/">여기</a>를 참고

## 2-5. Kernel Trick (커널 트릭)

- SVM은 "고차원" 공간으로 사상한 후 내적을 수행함
- 하지만 고차원 공간으로 사상한 후 내적을 구하는 것은 매우 복잡함
- 현재 차원에서 동일한 효과를 거두는 커널 함수를 사용하는 "커널 트릭"을 사용
- Mercer의 이론에 따르면 다음 조건을 만족하면 커널 트릭 함수로 대체 가능 <br/>
<br/>
$\int K(u,v) \psi (u) \psi (v) dxdy \leq 0$ <br/>
$where \int \psi(x)^2dx \leq 0 $ <br/>

__자주 쓰이는 커널 트릭 함수__ <br/>
- Polynomial kernel $K(a, b) = (a^Tb+1)^d$
- Radial kernel $K(a, b) = \exp (- \lambda ||a-b||^2)$
- Sigmoid kernel $K(a, b) = tanh(ka^Tb + \theta )$

### 2-6. 예제

In [6]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.svm import SVC

In [7]:
# 데이터 불러오기
bc = load_breast_cancer()

# 데이터프레임으로 변환 (데이터 확인용으로 사용함)
df = pd.DataFrame(bc.data, columns=bc.feature_names)
df['target'] = pd.Series(bc.target)

df.head()

,mean radius,mean texture,mean perimeter,mean area,mean smoothness,mean compactness,mean concavity,mean concave points,mean symmetry,mean fractal dimension,...,worst texture,worst perimeter,worst area,worst smoothness,worst compactness,worst concavity,worst concave points,worst symmetry,worst fractal dimension,target
0,17.99,10.38,122.80,1001.0,0.11840,0.27760,0.3001,0.14710,0.2419,0.07871,...,17.33,184.60,2019.0,0.1622,0.6656,0.7119,0.2654,0.4601,0.11890,0
1,20.57,17.77,132.90,1326.0,0.08474,0.07864,0.0869,0.07017,0.1812,0.05667,...,23.41,158.80,1956.0,0.1238,0.1866,0.2416,0.1860,0.2750,0.08902,0
2,19.69,21.25,130.00,1203.0,0.10960,0.15990,0.1974,0.12790,0.2069,0.05999,...,25.53,152.50,1709.0,0.1444,0.4245,0.4504,0.2430,0.3613,0.08758,0
3,11.42,20.38,77.58,386.1,0.14250,0.28390,0.2414,0.10520,0.2597,0.09744,...,26.50,98.87,567.7,0.2098,0.8663,0.6869,0.2575,0.6638,0.17300,0
4,20.29,14.34,135.10,1297.0,0.10030,0.13280,0.1980,0.10430,0.1809,0.05883,...,16.67,152.20,1575.0,0.1374,0.2050,0.4000,0.1625,0.2364,0.07678,0


In [10]:
# train / test set으로 분리
X_train, X_test, y_train, y_test = train_test_split(bc.data, bc.target, test_size=0.3, random_state=42)

svm = SVC() # 커널 함수 default: RBF

# 학습
svm.fit(X_train, y_train)

# 예측
y_pred = svm.predict(X_test)

In [11]:
# 정확도
print('Accuracy: ', round(accuracy_score(y_test, y_pred), 4)*100, '%')

Accuracy:  93.57 %


# 3. 과제

Kaggle의 <a href="https://www.kaggle.com/balaka18/email-spam-classification-dataset-csv"> email spam classification dataset</a>을 Naive Bayes와 SVM으로 각각 학습 시킨 후 예측한 결과의 정확도 계산 <br/>
- EDA, 전처리는 자유
- 하이퍼파라미터 변경도 자유

In [12]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.svm import SVC
from sklearn.naive_bayes import GaussianNB # 다른 나이브 베이즈 모델을 써도 무관함
from sklearn.metrics import accuracy_score

In [13]:
email = pd.read_csv('emails.csv', index_col='Email No.')
email.head()

,the,to,ect,and,for,of,a,you,hou,in,...,connevey,jay,valued,lay,infrastructure,military,allowing,ff,dry,Prediction
Email No.,,,,,,,,,,,,,,,,,,,,,
Email 1,0,0,1,0,0,0,2,0,0,0,...,0,0,0,0,0,0,0,0,0,0
Email 2,8,13,24,6,6,2,102,1,27,18,...,0,0,0,0,0,0,0,1,0,0
Email 3,0,0,1,0,0,0,8,0,0,4,...,0,0,0,0,0,0,0,0,0,0
Email 4,0,5,22,0,5,1,51,2,10,1,...,0,0,0,0,0,0,0,0,0,0
Email 5,7,6,17,1,5,2,57,0,9,3,...,0,0,0,0,0,0,0,1,0,0


In [14]:
# 데이터셋을 X와 y로 분리
X = email.drop('Prediction', axis=1)
y = email['Prediction']

# 데이터셋을 train/test set으로 분리
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)


In [ ]:
# Naive Bayes 모델

In [ ]:
# SVM

In [ ]:
# 정확도 계산